# Model based grading

In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [2]:
# Helper functions
def add_messages(messages, text, role="user"):
    message = {"role": role, "content": text}
    messages.append(message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [3]:
# Function to generate a new dataset
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "json" or "python" or "regex"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_messages(messages, prompt, role="user")
    add_messages(messages, "```json", role="assistant")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [ ]:
# Generate the dataset and write it to 'dataset.json'
dataset = generate_dataset()
with open("dataset.json", "w") as f:
    json.dump(dataset, f, indent=4)

In [5]:
# Function to grade a test case + output using a model
def grade_by_model(test_case, output):
    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>

Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_messages(messages, eval_prompt, role="user")
    add_messages(messages, "```json", role="assistant")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [6]:
# Passes a test case into Claude
def run_prompt(test_case):
    prompt = f"""
Please solve the following task:

{test_case["task"]}
"""

    messages = []
    add_messages(messages, prompt, role="user")
    output = chat(messages)
    return output

In [7]:
# Function to execute a single test case and grade the output
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)

    model_grade = grade_by_model(test_case, output)
    score = model_grade["score"]
    reasoning = model_grade["reasoning"]

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
        "reasoning": reasoning,
    }

In [8]:
from statistics import mean


def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [9]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average score: 7.333333333333333


In [ ]:
print(json.dumps(results, indent=4))

[
  {
    "output": "# AWS S3 Bucket Name Parser\n\nHere's a solution to parse the bucket name from an S3 URI:\n\n```python\ndef parse_s3_bucket_name(s3_uri: str) -> str:\n    \"\"\"\n    Extract the bucket name from an S3 URI.\n    \n    Args:\n        s3_uri: S3 URI in format 's3://bucket-name/key/path'\n        \n    Returns:\n        The bucket name\n        \n    Raises:\n        ValueError: If the URI format is invalid\n    \"\"\"\n    if not s3_uri.startswith('s3://'):\n        raise ValueError(\"S3 URI must start with 's3://'\")\n    \n    # Remove the 's3://' prefix\n    remainder = s3_uri[5:]\n    \n    # Extract bucket name (everything before the first '/')\n    bucket_name = remainder.split('/')[0]\n    \n    if not bucket_name:\n        raise ValueError(\"Bucket name cannot be empty\")\n    \n    return bucket_name\n\n\n# Test cases\nif __name__ == \"__main__\":\n    test_cases = [\n        \"s3://my-bucket/path/to/file.txt\",\n        \"s3://bucket-name/key/path\",\n     